In [53]:
import pandas as pd

def clean_data(df):
    # Alterar o tipo para string
    df['idPessoa'] = df['idPessoa'].astype(str).str.zfill(11)
    return df

# Variável carregada 'df' do URI: /home/maicon/git/esjud/integracao-emeron-moodle/_SELECT_Inscricao_idInscricao_InscricaoAluno_idPessoa_Pessoa_nom_202405201404.csv
df = pd.read_csv(r'dataset/_SELECT_Inscricao_idInscricao_InscricaoAluno_idPessoa_Pessoa_nom_202405201404.csv')

df_clean = clean_data(df.copy())

In [54]:
def clean_data_mdl_user(df):
    # Filtrar linhas com base na coluna: 'username'
    df = df.set_index('username')
    df = df.filter(regex="(^\\d+)",axis=0)
    return df

# Variável carregada 'df' do URI: /home/maicon/git/esjud/integracao-emeron-moodle/mdl_user.csv
df = pd.read_csv(r'dataset/mdl_user-v2.csv')

df_mdl_user = clean_data_mdl_user(df.copy()).reset_index()


In [55]:
df_clean['username'] = df_clean['idPessoa']

In [56]:
df_merged = df_clean.merge(df_mdl_user,on='username', how='outer')

In [57]:
df_merged.columns

Index(['idInscricao', 'idPessoa', 'nome', 'celular01', 'nome.1', 'flInscrito',
       'inscricaoData', 'inscricaoHora', 'idFormacao', 'flCertMoodle',
       'certMoodleId', 'username', 'id', 'auth', 'confirmed', 'policyagreed',
       'deleted', 'suspended', 'mnethostid', 'password', 'idnumber',
       'firstname', 'lastname', 'email', 'emailstop', 'phone1', 'phone2',
       'institution', 'department', 'address', 'city', 'country', 'lang',
       'calendartype', 'theme', 'timezone', 'firstaccess', 'lastaccess',
       'lastlogin', 'currentlogin', 'lastip', 'secret', 'picture',
       'description', 'descriptionformat', 'mailformat', 'maildigest',
       'maildisplay', 'autosubscribe', 'trackforums', 'timecreated',
       'timemodified', 'trustbitmask', 'imagealt', 'lastnamephonetic',
       'firstnamephonetic', 'middlename', 'alternatename', 'moodlenetprofile'],
      dtype='object')

Faço uma cópia com as colunas ['idPessoa','username','firstname', 'email', 'auth', 'firstaccess] e removo os duplicados
- com a coluna 'firstaccess' eu consigo ver três situações:
- não cadastrados no emeron
- não cadastrados no moodle
- usuários que nunca acessaram (que estão com valor 0)

In [58]:
df_analise = df_merged[['idPessoa','username','firstname', 'email', 'auth', 'firstaccess']].copy().drop_duplicates()

Função para converter as colunas de timestamp para datetime, os valores não reconhecidos como data serão ignorados

In [59]:
def timestamp_to_datetime_series(serie:pd.Series):
    return pd.to_datetime(serie, unit='s', errors='coerce')

In [60]:
# converte uma nova coluna
df_analise['firstaccess_datetime'] = timestamp_to_datetime_series(df_analise['firstaccess'].copy())

In [62]:
df_analise.columns


Index(['idPessoa', 'username', 'firstname', 'email', 'auth', 'firstaccess',
       'firstaccess_datetime'],
      dtype='object')

In [ ]:
df_analise = df_analise.drop('firstaccess', axis=1)

In [ ]:
df_analise = pd.concat([df_analise, df_merged[['lastaccess','lastlogin']]], axis=1)

In [ ]:
df_analise['lastaccess'] = timestamp_to_datetime_series(df_analise['lastaccess'].copy())

In [ ]:
df_analise['lastlogin'] = timestamp_to_datetime_series(df_analise['lastlogin'].copy())

In [ ]:
df_merged = df_merged[df_merged['nome.1'].str.contains("Artifi", regex=False, na=False)]

In [ ]:
df_nulos_no_emeron = df_merged[df_merged['idPessoa'].isnull() == True]

In [ ]:
df_nulos_no_emeron